## Stimulus coding with HDDMRegression

Note: This tutorial is more advanced. If you are just starting you might want
to head to the :ref:`demo <chap_demo>` instead.

In some situations it is useful to fix the magnitude of parameters
across stimulus types while also forcing them to have different
directions. For example, an independent variable could influence both
the drift rate ``v`` and the response bias ``z``. A specific example is an
experiment on face-house discrimination with different difficulty
levels, where the drift-rate is smaller when the task is more
difficult and where the bias to responding house is larger when the
task is more difficult.  One way to analyze the effect of difficulty
on drift rate and bias in such an experiment is to estimate one drift
rate ``v`` for each level, and a response bias ``z`` such that the bias for
houses-stimuli is ``z`` and the bias for face stimuli is ``1-z`` (``z = .5``
for unbiased decisions in ``HDDM``).

The following example describes how to generate simulated data for
such an experiment, how to set up the analysis with ``HDDMRegression``,
and compares true parameter values with those estimated with
``HDDMRegression``.

### Model Recovery Test for HDDMRegression

The test is performed with simulated data for an experiment with one
independent variable with three levels (e.g. three levels of
difficulty) which influence both drift rate ``v`` and bias ``z``. Responses
are "accuracy coded", i.e. correct responses are coded ``1`` and incorrect
responses ``0``. Further, stimulus coding of the parameter ``z`` is
implemented. "stimulus coding" of ``z`` means that we want to fit a model
in which the magnitude of the bias is the same for the two stimuli,
but its direction "depends on" the presented stimulus (e.g. faces or
house in a face-house discrimination task). Note that this does not
mean that we assume that decision makers adjust their bias after
having seen the stimulus. Rather, we want to measure response-bias (in
favor of face or house) while assuming the same drift rate for both
stimuli. We can achieve this for accuracy coded data by modeling the
bias as moved towards the correct response boundary for one stimulus
(e.g. ``z = .6`` for houses) and away from the correct response boundary
for the other stimulus (``1-z = .4`` for faces).

First, we need to import the required python modules.

In [1]:
# update kabuki to make sure compatibility
# !pip uninstall kabuki -y
# !pip install git+https://gitee.com/epool/kabuki.git

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from patsy import dmatrix  # for generation of (regression) design matrices
from pandas import Series  # to manipulate data-frames generated by hddm
%matplotlib inline
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import hddm
import kabuki
print("The current version of kabuki is: ", kabuki.__version__)
print("The current version of HDDM is: ", hddm.__version__)

The current version of kabuki is:  0.6.5RC4
The current version of HDDM is:  0.8.0


We save the output of stdout to the file ``ModelRecoveryOutput.txt``.


In [3]:
import sys

sys.stdout = open("ModelRecoveryOutput.txt", "w")

[                  0%                  ] 2 of 2000 complete in 0.5 sec
[                  0%                  ] 4 of 2000 complete in 1.3 sec
[                  0%                  ] 6 of 2000 complete in 2.1 sec
[                  0%                  ] 8 of 2000 complete in 2.9 sec
[                  0%                  ] 10 of 2000 complete in 3.7 sec
[                  0%                  ] 12 of 2000 complete in 4.5 sec
[                  0%                  ] 14 of 2000 complete in 5.3 sec
[                  0%                  ] 16 of 2000 complete in 6.0 sec
[                  0%                  ] 18 of 2000 complete in 6.9 sec
[                  1%                  ] 20 of 2000 complete in 7.6 sec
[                  1%                  ] 22 of 2000 complete in 8.4 sec
[                  1%                  ] 24 of 2000 complete in 9.2 sec
[                  1%                  ] 26 of 2000 complete in 10.0 sec
[                  1%                  ] 28 of 2000 complete in 10.

#### Creating simulated data for the experiment

Next we set the number of subjects and the number of trials per level
for the simulated experiment

In [4]:
n_subjects = 10
trials_per_level = 150  # and per stimulus

Next we set up parameters of the drift diffusion process for the three
levels and the first stimulus. As desribed earlier ``v`` and ``z`` change
accross levels.

In [5]:
level1a = {"v": 0.3, "a": 2, "t": 0.3, "sv": 0, "z": 0.5, "sz": 0, "st": 0}
level2a = {"v": 0.4, "a": 2, "t": 0.3, "sv": 0, "z": 0.6, "sz": 0, "st": 0}
level3a = {"v": 0.5, "a": 2, "t": 0.3, "sv": 0, "z": 0.7, "sz": 0, "st": 0}

Now we generate the data for stimulus A

In [6]:
data_a, params_a = hddm.generate.gen_rand_data(
    {"level1": level1a, "level2": level2a, "level3": level3a},
    size=trials_per_level,
    subjs=n_subjects,
)

Next come the parameters for the second stimulus, where ``v`` is the same
as for the first stimulus. This is different for ``z``. In particular:
``z(stimulus_b) = 1 - z(stimulus_a)``. As a result, responses are
altogether biased towards responding A. Because we use accuracy coded
data, stimulus A is biased towards correct responses, and stimulus B
towards incorrect responses. 

In [7]:
level1b = {"v": 0.3, "a": 2, "t": 0.3, "sv": 0, "z": 0.5, "sz": 0, "st": 0}
level2b = {"v": 0.4, "a": 2, "t": 0.3, "sv": 0, "z": 0.4, "sz": 0, "st": 0}
level3b = {"v": 0.5, "a": 2, "t": 0.3, "sv": 0, "z": 0.3, "sz": 0, "st": 0}

Now we generate the data for stimulus B

In [8]:
data_b, params_b = hddm.generate.gen_rand_data(
    {"level1": level1b, "level2": level2b, "level3": level3b},
    size=trials_per_level,
    subjs=n_subjects,
)

We add a column to the ``DataFrame`` identifying stimulus A as 1 and stimulus B as 2.


In [9]:
data_a["stimulus"] = Series(np.ones((len(data_a))), index=data_a.index)
data_b["stimulus"] = Series(np.ones((len(data_b))) * 2, index=data_a.index)

Now we merge the data for stimulus A and B

In [10]:
mydata = pd.concat([data_a,data_b])
mydata.reset_index(drop=True,inplace=True)
mydata.head()

,rt,response,subj_idx,condition,stimulus
0,0.422029,1.0,0,level1,1.0
1,0.920318,1.0,0,level1,1.0
2,2.323944,1.0,0,level1,1.0
3,0.546244,1.0,0,level1,1.0
4,1.717238,1.0,0,level1,1.0


#### Setting up the HDDM regression model

Next we need to ensure that the bias is ``z`` for one stimulus and ``1-z``
for the other stimulus.  This is implemented here for all stimulus A trials
and -1 for stimulus B trials. We use the ``patsy`` command ``dmatrix`` to
generate such an array from the stimulus column of our simulated data


In [11]:
def z_link_func(x, data=mydata):
    stim = np.asarray(
        dmatrix("0 + C(s, [[0], [1]])", {"s": data.stimulus.loc[x.index]})
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = stim - x
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

(NOTE: earlier versions of this tutorial suggested applying an inverse logit
link function to the regression, but this should no longer be used given changes to the prior 
on the intercept.) 
Also depending on your python version, the above code may give you errors and you can try this instead:


In [12]:
def z_link_func(x, data=mydata):
    stim = np.asarray(
        dmatrix(
            "0 + C(s, [[0], [1]])",
            {"s": data.stimulus.loc[x.index]},
            return_type="dataframe",
        )
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = np.subtract(stim, x.to_frame())
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

Now we set up the regression models for ``z`` and ``v`` and also include the
link functions The relevant string here used by ``patsy`` is '1 +
C(condition)'. This will generate a design matrix with an intercept
(that's what the '1' is for) and two dummy variables for remaining
levels. (The column in which the levels are coded has the default name
'condition'):

In [13]:
z_reg = {"model": "z ~ 1 + C(condition)", "link_func": z_link_func}

For ``v`` the link function is simply ``x = x``, because no transformations is
needed. [However, you could also analyze this experiment with response
coded data. Then you would not stimulus code ``z`` but ``v`` and you would
have to multiply the ``v`` for one condition with ``-1``, with a link function
like the one for ``z`` above, but with out the additional logit transform
]

In [14]:
v_reg = {"model": "v ~ 1 + C(condition)", "link_func": lambda x: x}

Now we can finally put the regression description for the hddm model
together. The general template for this is ``[{'model': 'outcome_parameter ~ patsy_design_string', 'link_func': your_link_function }, {...}, ...]``


In [15]:
reg_descr = [z_reg, v_reg]

The last step before running the model is to construct the complete hddm regression model by adding data etc.

In [16]:
m_reg = hddm.HDDMRegressor(mydata, reg_descr, include=["v", "a", "t", "z"])

In [17]:
# m_reg.sample(2000, burn=100)
save_name = "model_fitted/hddmregressor_example"
m_reg.sample(
    2000, burn=500, 
    chains=4, 
    return_infdata = True, 
    loglike=True,
    sample_prior=True,
    ppc=True,
    save_name = save_name
)

/opt/conda/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2309: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2309: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2309: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2309: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> prior
	> observed_data

Now we start the model, and wait for a while (you can go and get
several coffees, or read a paper). 

In [18]:
# m_reg.infdata

#### Comparing generative and recovered model parameters

First we print the model stats

In [19]:
# m_reg.gen_stats()

import arviz as az
az.summary(m_reg.infdata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,1.972,0.059,1.868,2.084,0.001,0.001,4755.0,3072.0,1.0
a_std,0.175,0.054,0.099,0.279,0.001,0.001,2588.0,2118.0,1.0
a_subj.0,1.989,0.032,1.926,2.048,0.001,0.000,3344.0,3929.0,1.0
a_subj.1,2.032,0.037,1.964,2.100,0.001,0.001,2475.0,3525.0,1.0
a_subj.2,2.031,0.035,1.966,2.099,0.001,0.000,3058.0,3699.0,1.0
a_subj.3,2.301,0.041,2.220,2.375,0.001,0.001,2682.0,3370.0,1.0
a_subj.4,1.950,0.031,1.893,2.010,0.001,0.000,3328.0,4141.0,1.0
a_subj.5,1.986,0.035,1.925,2.055,0.001,0.000,2502.0,3632.0,1.0
a_subj.6,1.890,0.030,1.832,1.947,0.001,0.000,3135.0,3757.0,1.0
a_subj.7,1.860,0.035,1.798,1.927,0.001,0.000,2608.0,3357.0,1.0


Lets first look at ``v``. For ``level1`` this is just the
intercept. The value of ``.33`` is in the ball park of the true value
of ``.3``. The fit is not perfect, but running a longer chain might
help (we are ignoring sophisticated checks of model convergence for
this example here). To get the values of ``v`` for levels 2 and 3, we
have to add the respective parameters (``0.11`` and ``.26``) to the
intercept value. The resulting values of  are again
close enough to the true values of ``.4`` and ``.5``. The ``z_Intercept``
value of .47 is close tothe true value of ``.5``, and the level 2 and level 3
offsets are also close (.47 - .001 = .44 and .47 -0.02 = 0.45).   In sum,
``HDDMRegression`` easily recovered the right order of the parameters
``z``. The recovered parameter values are also close to the true
parameter values, and this was only for a single subject fit.
Parameter estimates are improved with more subjects. 